In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import scipy.stats as st
import sys 
import datetime as dt
import time
import gc
gc.collect()

LESSON_DATA_FOLDER = './data/'
#http://www.imdb.com/interfaces/

In [2]:
from helper.parallel_util import ParallelUtil
from helper.parsers import Utils
from helper.file_mapping import FILE_MAPPINGS
from helper.imdb_sqllite_db import IMDBSQLLite

imdb_db = IMDBSQLLite();
imdbConn, imdbCurs = imdb_db.get()

IMDB SQLLite Database: ./data/imdb.sqllite


In [3]:
#clusters = ParallelUtil('./data/parallel.config.tsv')      
#clusters.info()

In [4]:
import read_data as imdbImport

In [5]:
#importLib.mergeAll()

#imdbImport.merge_title_ratings(None, 10);
#imdbImport.merge_names_principals(None, 10);
#importLib.merge_names_principals(''.join([LESSON_DATA_FOLDER, 'merged.names.principals.tsv']), None)

In [6]:
'''
    #@dview.parallel(block=True)
    def f(group):
        #print (len(group))

        #group = pandas.DataFrame(group.to_dict('dict'))

        group = group.set_index('tconst')    
        data = {'directors': group.to_json(orient='records')}
        #data = {'directors': group.to_dict('dict')}
        #print (group.index.values , directors)
        return pandas.DataFrame(data=data, index = group.index.values)  



#title_grouped_directors# = 
#title_grouped_directors.reset_index().drop(['level_1'], axis= 1)
#pd.DataFrame(title_grouped_directors).to_json(orient='records')
#title_grouped_directors.reset_index(level='tconst')
'''

"\n    #@dview.parallel(block=True)\n    def f(group):\n        #print (len(group))\n\n        #group = pandas.DataFrame(group.to_dict('dict'))\n\n        group = group.set_index('tconst')    \n        data = {'directors': group.to_json(orient='records')}\n        #data = {'directors': group.to_dict('dict')}\n        #print (group.index.values , directors)\n        return pandas.DataFrame(data=data, index = group.index.values)  \n\n\n\n#title_grouped_directors# = \n#title_grouped_directors.reset_index().drop(['level_1'], axis= 1)\n#pd.DataFrame(title_grouped_directors).to_json(orient='records')\n#title_grouped_directors.reset_index(level='tconst')\n"

In [7]:
'''
print (time.strftime("%Y-%m-%d %H:%M:%S"))
clusters = PARALLEL_CLUSTERS()
dview = clusters[:]
dview.scatter("scview", title_grouped_directors.apply(f))
#dview['scview']
title_grouped_directors = pd.concat([i for i in dview["scview"]])
print (time.strftime("%Y-%m-%d %H:%M:%S"))
#title_grouped_directors = title_grouped_directors.reset_index().drop(['level_1'], axis= 1)
'''

'\nprint (time.strftime("%Y-%m-%d %H:%M:%S"))\nclusters = PARALLEL_CLUSTERS()\ndview = clusters[:]\ndview.scatter("scview", title_grouped_directors.apply(f))\n#dview[\'scview\']\ntitle_grouped_directors = pd.concat([i for i in dview["scview"]])\nprint (time.strftime("%Y-%m-%d %H:%M:%S"))\n#title_grouped_directors = title_grouped_directors.reset_index().drop([\'level_1\'], axis= 1)\n'

In [8]:
#imdbImport.mergeAll(1000);
imdbImport.mergeAllToTable(imdbConn, 1000);

Start: 2018-03-12 12:16:49
['tconst', 'averageRating', 'numVotes']
End: 2018-03-12 12:16:49
Start: 2018-03-12 12:16:49
['tconst', 'titleType', 'primaryTitle', 'originalTitle', 'isAdult', 'startYear', 'endYear', 'runtimeMinutes', 'genres']
End: 2018-03-12 12:16:49
wikipedia Done: 2018-03-12 12:16:49
Merge done: 2018-03-12 12:16:49
To TABLE merged_titles_ratings done: 2018-03-12 12:16:50
{'tconst': dtype('O'), 'titleType': dtype('O'), 'primaryTitle': dtype('O'), 'originalTitle': dtype('O'), 'isAdult': dtype('bool'), 'startYear': dtype('int64'), 'endYear': dtype('O'), 'runtimeMinutes': dtype('float64'), 'genres': dtype('O'), 'wikiTitle': dtype('O'), 'averageRating': dtype('float64'), 'numVotes': dtype('float64')}
Start: 2018-03-12 12:16:50
['nconst', 'primaryName', 'birthYear', 'deathYear', 'primaryProfession', 'knownForTitles']
End: 2018-03-12 12:16:50
Alive Done: 2018-03-12 12:16:50
wikipedia Done: 2018-03-12 12:16:50
Start: 2018-03-12 12:16:50
['tconst', 'ordering', 'nconst', 'category